This is where I am planning the ananlysis of the Education Library data set. I'll make a mark-down comment block with each of my questions/hypotheses and then we will find the answers and graph the results. This analysis will focus on how often books have circulated. 

What is the average amount of time since a book has been checked out? Make a histogram of how many years it has been since each book was checked out. (<1 years, <2 years, etc)

In [ ]:
#code here

In [ ]:
What percentage of books have never been checked out?

In [ ]:
#code here

What percentage of the collection has ciruclated in the last 1, 5, 10, and 20 years? let's make a little bar graph of what percentage of the books have been checked out in that time frame. 

In [ ]:
#code here

How many books have been checked out 0, 1, 2, 3, 4, 5, or 5+ times? make a pie chart sorting the books into these categories.

In [ ]:
#code here

What are the top 10 most circulated books in the data base? Do any of those 10 books have the same author? What is the average year of publication for these top 10 books?

In [ ]:
#code here

Are newer books checked out more freuqeuntly? Make a 2D plot showing cat creat year vs. avg number of check outs per year for each book. Maybe fit it with a linear regression or something.

In [ ]:
#code here